# Projeto 1 - Ciência dos Dados

Nome: Alexandre Cury

Nome: Celso H A Diniz

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [105]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import warnings
import re
import string
warnings.filterwarnings("ignore") 

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [106]:
import os
#ler os data coletados
# -*- coding: utf-8 -*-
filename = "PS5_classificado-2.xlsx"
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo PS5_classificado-2.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [107]:
data = pd.read_excel("PS5_classificado-2.xlsx")
data.head()

,Treinamento,Classificado
0,fortnite no playstation 5 no dia do lançamento...,0
1,o anúncio ocorre um dia após a sony brasil ter...,0
2,o playstation 5 é bonito? é! quero um? sim! te...,1
3,@playstation_br vou ao paraguai e trazer para ...,1
4,@stefano_poke nao viaja pelo amor de deus....\...,1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido foi o Playstation 5, que teve seu lançamento recente e coinciencidente com a data de publição do projeto, e o anúncio do seu preço no Brasil.
O tweet considerava relevante continha a indicação sobre a inteção ou não intenção de compra do produto, considerando o preço pelo o qual será vendido no Brasil

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [108]:
# definir 0 como irrelevante e 1 como relevante, variavel categorica

data["Classificado"] = data["Classificado"].astype('category')
data["Classificado"].cat.categories = ['relevante', 'irrelevante']
data.head()

,Treinamento,Classificado
0,fortnite no playstation 5 no dia do lançamento...,relevante
1,o anúncio ocorre um dia após a sony brasil ter...,relevante
2,o playstation 5 é bonito? é! quero um? sim! te...,irrelevante
3,@playstation_br vou ao paraguai e trazer para ...,irrelevante
4,@stefano_poke nao viaja pelo amor de deus....\...,irrelevante


In [109]:
def clean(tweet):
    return ''.join([i if ord(i) < 128 else '' for i in tweet])
data["Treinamento"] = data["Treinamento"].apply(clean)
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.lower())
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace('.',' '))
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace('\n',' '))
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace('?',' '))
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace('!',' '))
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace('"',' '))
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace(';',' '))
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace('#',' '))
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace('&amp',' '))
data["Treinamento"] = data["Treinamento"].apply(lambda x: x.replace(',',' '))
data.Classificado.value_counts(True)

relevante      0.703333
irrelevante    0.296667
Name: Classificado, dtype: float64

In [110]:
filtro_data = data.Treinamento.str.cat()
filtro_data = re.sub("[" + string.punctuation + "]", "", filtro_data.lower())
palavras = filtro_data.split() # lista de todas as palavras 



In [111]:
total = len(palavras) #numero total de palvras na base de dados
num_rel = data.Classificado.value_counts(True)[0] # porcentagem de items relevantes
num_irr = data.Classificado.value_counts(True)[1] # porcentagens de itens irrelevantes
palavras_relevantes = total*data.Classificado.value_counts(True)[0]
palavras_irrelevantes = total*data.Classificado.value_counts(True)[1]
print("O total de palavras na planilha é: ",total)
print(52*"-")
print("O total de palavras em relevantes é: ", palavras_relevantes.round(3))
print(52*"-")
print("O total de palavras em irrelevantes é: ", palavras_irrelevantes.round(3))
print(52*"-")
print("A probabilidade de palavras em relevantes é: ",num_rel.round(3))
print(52*"-")
print("A probabilidade de palavras em relevantes é: ",num_irr.round(3))

O total de palavras na planilha é:  5622
----------------------------------------------------
O total de palavras em relevantes é:  3954.14
----------------------------------------------------
O total de palavras em irrelevantes é:  1667.86
----------------------------------------------------
A probabilidade de palavras em relevantes é:  0.703
----------------------------------------------------
A probabilidade de palavras em relevantes é:  0.297


# Tabelas relevantes

## Tabela absoluta relevante

In [112]:
relevante = data["Classificado"] == 'relevante' # pegar somente dados classificados como relevante
data_relevante = data.loc[relevante, :] # filtrar no dataframe
tweet_relevante = pd.Series(data_relevante.Treinamento) #gera uma series de dados relevantes
tweet_relevante = pd.DataFrame(tweet_relevante) #transforma a series em um dataframe
tweet_relevante["Treinamento"] = tweet_relevante["Treinamento"].apply(clean)
palavras_relevantes = tweet_relevante.Treinamento.str.cat() # transforma o dataframe em uma lista de palavras
palavras_relevantes = palavras_relevantes.split()
palavras_relevantes = pd.Series(palavras_relevantes)


abs_relevante = palavras_relevantes.value_counts()
total_relevante = abs_relevante.sum()
print(abs_relevante)
print("Numero total de palavras relevantes: ", total_relevante)




5              174
playstation    141
https://t      141
no             123
o              113
              ... 
voc              1
aumenta          1
padro            1
cobrir           1
creio            1
Length: 1268, dtype: int64
Numero total de palavras relevantes:  4059


## Tabela Relativa

In [113]:
rel_relevante = palavras_relevantes.value_counts(True)
print(rel_relevante)
print(rel_relevante.sum())

5              0.042868
playstation    0.034738
https://t      0.034738
no             0.030303
o              0.027839
                 ...   
voc            0.000246
aumenta        0.000246
padro          0.000246
cobrir         0.000246
creio          0.000246
Length: 1268, dtype: float64
1.0


# Tabelas Irrelevantes

## Tabela absoluta

In [114]:
irrelevante = data["Classificado"] == 'irrelevante' # pegar somente dados classificados como relevante
data_irrelevante = data.loc[irrelevante, :] # filtrar no dataframe
tweet_irrelevante = pd.Series(data_irrelevante.Treinamento) #gera uma series de dados relevantes
tweet_irrelevante = pd.DataFrame(tweet_irrelevante) #transforma a series em um dataframe
tweet_irrelevante["Treinamento"] = tweet_irrelevante["Treinamento"].apply(clean)
palavras_irrelevantes = tweet_irrelevante.Treinamento.str.cat() # transforma o dataframe em uma lista de palavras
palavras_irrelevantes = palavras_irrelevantes.split()
palavras_irrelevantes = pd.Series(palavras_irrelevantes)


abs_irrelevante = palavras_irrelevantes.value_counts()
total_irrelevante = abs_irrelevante.sum()
print(abs_irrelevante)
print("Numero total de palavras irrelevantes: ", total_irrelevante)






5              91
playstation    72
o              61
eu             40
um             39
               ..
pack            1
sony            1
rgos            1
srio            1
disso           1
Length: 606, dtype: int64
Numero total de palavras irrelevantes:  1613


## Tabela relativa

In [115]:
## Tabela relativa
rel_irrelevante = palavras_irrelevantes.value_counts(True)
print(rel_irrelevante)
print(rel_irrelevante.sum())

5              0.056417
playstation    0.044637
o              0.037818
eu             0.024799
um             0.024179
                 ...   
pack           0.000620
sony           0.000620
rgos           0.000620
srio           0.000620
disso          0.000620
Length: 606, dtype: float64
0.9999999999999999


# Dashboard: panorama inicial 

In [116]:
print("Numero total de palavras relevantes é: ", total_relevante)
print(55*"-")
print("Numero total de palavras irrelevantes é: ", total_irrelevante)
print(55*"-")
total2 = total_relevante+total_irrelevante # apos o filtro

print("Numero total de palavras apos o filtro/ajustes é: ",  total2)

Numero total de palavras relevantes é:  4059
-------------------------------------------------------
Numero total de palavras irrelevantes é:  1613
-------------------------------------------------------
Numero total de palavras apos o filtro/ajustes é:  5672


# Probabilidades

In [117]:
#probabilidade de ser relevante ou irrelevante

prob_rel = total_relevante/total2
prob_irr = total_irrelevante/total2
checagem = prob_rel+prob_irr
print("A probabilidade de ser relevante é: ", prob_rel) # palavras
print("A probabilidade de ser irrelevante é: ", prob_irr) # palavras
print("Checagem: ",checagem)




A probabilidade de ser relevante é:  0.715620592383639
A probabilidade de ser irrelevante é:  0.28437940761636105
Checagem:  1.0


# Classificador

In [118]:
prob_tweet_rel = 1
prob_tweet_irr = 1



lista_rel = []
lista_irr = []

a = "relevante"
b = "irrelevante"

for tweet in data.Treinamento:
    tweet = clean(tweet)
    tweet = tweet.split()
    
    for i in range(len(tweet)):
        if tweet[i] in rel_relevante:
            prob_tweet_rel *= (abs_relevante[tweet[i]]+1)/(total_relevante+total2)
        else:
            prob_tweet_rel *= (0+1)/(total_relevante+total2)

        
        if tweet[i] in rel_irrelevante:
            prob_tweet_irr *= (abs_irrelevante[tweet[i]]+1)/(total_irrelevante+total2)
        else:
            prob_tweet_irr *= (0+1)/(total_irrelevante+total2)
            



            if (prob_tweet_rel*prob_rel) > (prob_tweet_irr*prob_irr):

                lista_rel.append("relevante")

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [119]:
total_classificado = len(data.Treinamento)
total_rel_classificado = len(lista_rel)
prob_rel_classificada = len(lista_rel)/len(data.Treinamento)
prob_irr_classificada = 1-((len(lista_rel)/len(data.Treinamento)))



print("o numero total de tweets no classificador é:", total_classificado)
print(55*"-")
print("o numero de tweets classificados como relevantes é: ", total_rel_classificado)
print(55*"-")
print("a probabilidade dos tweets serem relevantes é: ", prob_rel_classificada)
print(55*"-")
print("a probabilidade dos tweets serem irrelevantes é: ", prob_irr_classificada)

o numero total de tweets no classificador é: 300
-------------------------------------------------------
o numero de tweets classificados como relevantes é:  27
-------------------------------------------------------
a probabilidade dos tweets serem relevantes é:  0.09
-------------------------------------------------------
a probabilidade dos tweets serem irrelevantes é:  0.91


# Comparação com o previsto

In [120]:
print("A probabilidade esperada de ser relevante é: ", prob_rel.round(2))
print(55*"-")
print("A probabilidade apos o classificador: ", prob_rel_classificada) 
print(55*"-")
print("Diferenca de: ", (prob_rel - prob_rel_classificada).round(2))
print(55*"-")
print("   ")
print("A probabilidade esperada de ser irrelevante é: ", prob_irr.round(2))
print(55*"-")
print("A probabilidade calculada pelo classificador: ", prob_irr_classificada) 
print(55*"-")
print("Diferenca de: ", (prob_irr_classificada - prob_irr).round(2))




A probabilidade esperada de ser relevante é:  0.72
-------------------------------------------------------
A probabilidade apos o classificador:  0.09
-------------------------------------------------------
Diferenca de:  0.63
-------------------------------------------------------
   
A probabilidade esperada de ser irrelevante é:  0.28
-------------------------------------------------------
A probabilidade calculada pelo classificador:  0.91
-------------------------------------------------------
Diferenca de:  0.63


___
### Concluindo


Podemos reparar que o classificador divergiu bastante do que era esperado,resultando em uma diferenca de 63%.
Isso se deve a alguns motivos como o fato do Playstation 5 aindao nao ter cido lançado, gerou muitas especulacoes, mesmo que grande parte delas fosse relevante para algumas pessoas. Outra possibilidade é que o gosto variado das pessoas em relacao a consoles varia muito e em muitos casos quem costuma jogar bastante tem mais de um console.

Um uso muito interessante do Classificador seria a classificao de mensagens de texto e apps de comunicação no geral. Coletando dados de uso do usuario, poderia ser facilmente feito um app que detecta que uma pessoas que enviou uma mensagem de texto utilizando o celular de um amigo/amiga, baseado no padrão e no historico da conversa entre os dois. Ou ainda, o app poderia evitar o envio me mensagens caso um numero fosse clonado, evitando que criminosos Peçam dinheiro, se passando pelo dono do telefone.

O próprio classificador nao pode ser usado para gerar mais amostras de treinamento, devido que o padrao tem chances de se repetir e os resultados podem sempre serem os mesmos.


Se for analizar os dados baseado na intenção de compra pelo preço, é muitos provavel que os brasileiros não comprariam o produto devido ao seu alto custo no pais, cerca de quase 5 vezes o salario minimo.

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**